In [1]:
import pandas as pd
import numpy as np

TRAIN = pd.read_csv("TRAIN.csv")
DEV   = pd.read_csv("DEV.csv")

def preprocess(df):
    df['target'] = df['aspectCategory']+ '-'+ df['polarity']
    df_dummy = pd.get_dummies(df, columns=['target'])
    return df[['id','text','target']]

train = preprocess(TRAIN)
dev   = preprocess(DEV)

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

X_train, X_dev, Y_train, Y_dev = train['text'], dev['text'], train['target'], dev['target']

vect = CountVectorizer()
X_train_bow = vect.fit_transform(X_train)
X_dev_bow = vect.transform(X_dev)

In [3]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(max_iter=300)
logreg.fit(X_train_bow, Y_train)

LogisticRegression(max_iter=300)

In [4]:
dev_pred = logreg.predict(X_dev_bow)

In [5]:
dev_pred_df = pd.DataFrame({
    'id':DEV.id,
    'aspectCategory': [e.split('-')[0] for e in dev_pred],
    'polarity': [e.split('-')[1] for e in dev_pred]
})

# Eval

In [7]:
from evaluate import reports

In [9]:
dev_pred_df.to_csv("dev_pred_df.csv", index=False)

In [10]:
reports("DEV.csv", "dev_pred_df.csv")

=== CLASSIFICATION : ASPECT ===
                class name  precision  recall  F1-score  support
0                     food      0.605   0.617     0.611      261
1                    price      0.220   0.127     0.161       71
2                  service      0.490   0.381     0.429      134
3                 ambience      0.304   0.177     0.224       96
4  anecdotes/miscellaneous      0.648   0.811     0.720      227
5                MACRO AVG      0.453   0.422     0.429      789
6                MICRO AVG      0.562   0.535     0.548      789 

=== CLASSIFICATION : SENTIMENT ===
  class name  precision  recall  F1-score  support
0   positive      0.718   0.875     0.789      440
1   negative      0.587   0.423     0.492      175
2    neutral      0.378   0.280     0.322      100
3   conflict      0.400   0.128     0.194       47
4  MACRO AVG      0.521   0.426     0.449      762
5  MICRO AVG      0.656   0.647     0.652      762 

=== CLASSIFICATION : OVERALL ===
              preci